In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/0j/h4x8twq57394t3xmh53ddyv40000gn/T/ipykernel_6071/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [6]:
# Let's first load the data from the CSV files provided and inspect the content to understand how to divide them into training and test sets.

import pandas as pd

# Load the provided datasets
train_data_path = "kg_train.csv"
test_data_path = "kg_test.csv"

# Read the data into pandas DataFrame
train_data = pd.read_csv(train_data_path, encoding='latin-1')
test_data = pd.read_csv(test_data_path, encoding='latin-1')

# Display the first few rows of each dataset
train_data.head(), test_data.head()


from sklearn.model_selection import train_test_split

# Split the training data into training and validation sets
train_set, validation_set = train_test_split(train_data, test_size=0.2, random_state=42)

# Display the shapes of the training and validation sets
train_set.shape, validation_set.shape



((4771, 2), (1193, 2))

## Data Preprocessing

In [7]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [8]:
import re

def clean_html(text):
    # Remove inline JavaScript/CSS by removing anything between <script>...</script> and <style>...</style>
    text = re.sub(r'<(script|style).*?>.*?(<\/\1>)', '', text, flags=re.DOTALL)

    # Remove HTML comments (anything between <!-- and -->)
    text = re.sub(r'<!--.*?-->', '', text)

    # Remove any remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    return text

# Sample HTML data for testing
sample_html = """
<html>
<head><script>var x = 10;</script><style>body {background-color: #fff;}</style></head>
<body>
<!-- This is a comment -->
<p>This is a paragraph.</p>
</body>
</html>
"""

cleaned_text = clean_html(sample_html)
cleaned_text  # Cleaned text output


'\n\n\n\n\nThis is a paragraph.\n\n\n'

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [9]:
def preprocess_text(text):
    # Remove special characters (everything except letters and spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove single characters
    text = re.sub(r'\b\w\b', '', text)
    
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove any prefixed 'b' (sometimes appears with byte string representation)
    text = re.sub(r'\bb\b', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Strip leading/trailing spaces
    return text.strip()

# Sample text for preprocessing
sample_text = "Hello, b2! This is a 100 test b. Let's remove $pecial char@cters!"
preprocessed_text = preprocess_text(sample_text)
preprocessed_text  # Preprocessed text output


'hello this is test lets remove pecial charcters'

## Now let's work on removing stopwords
Remove the stopwords.

In [10]:
import nltk
from nltk.corpus import stopwords

# Ensure stopwords are downloaded
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    # Remove stopwords from the text
    filtered_text = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_text)

# Sample text for removing stopwords
sample_text = "This is an example sentence to demonstrate the removal of stopwords."
text_without_stopwords = remove_stopwords(sample_text)
text_without_stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/keremsenler/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'example sentence demonstrate removal stopwords.'

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [11]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    # Lemmatize each word in the text
    lemmatized_text = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_text)

# Sample text for lemmatization
sample_text = "The boys are playing with the running dogs."
lemmatized_text = lemmatize_text(sample_text)
lemmatized_text


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/keremsenler/nltk_data...


'The boy are playing with the running dogs.'

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [14]:
# Combine ham and spam messages into one series
combined_messages = pd.concat([ham_messages, spam_messages])

# Fit the vectorizer on the combined messages
vectorizer = CountVectorizer(stop_words='english')
combined_bow = vectorizer.fit_transform(combined_messages)

# Separate the bow into ham and spam again
ham_bow = combined_bow[:len(ham_messages), :]
spam_bow = combined_bow[len(ham_messages):, :]

# Convert to DataFrames
ham_words = pd.DataFrame(ham_bow.toarray(), columns=vectorizer.get_feature_names_out())
spam_words = pd.DataFrame(spam_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Get the top 10 words in ham and spam
top_ham_words = ham_words.sum().sort_values(ascending=False).head(10)
top_spam_words = spam_words.sum().sort_values(ascending=False).head(10)

top_ham_words, top_spam_words


(state        1096
 pm            875
 secretary     626
 said          599
 obama         558
 president     537
 new           503
 house         473
 30            450
 2009          442
 dtype: int64,
 2e             10376
 money           5650
 br              5454
 2c              5249
 bank            4681
 account         4533
 nbsp            3618
 div             3001
 transaction     2842
 business        2658
 dtype: int64)

## Extra features

In [18]:
# Assuming you have a function preprocess_text() for text cleaning (from previous steps)

# Apply text preprocessing to both train and test data
train_data['preprocessed_text'] = train_data['text'].apply(preprocess_text)
test_data['preprocessed_text'] = test_data['text'].apply(preprocess_text)

# Now you can use 'preprocessed_text' for further processing
money_simbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

train_data['money_mark'] = train_data['preprocessed_text'].str.contains(money_simbol_list)*1
train_data['suspicious_words'] = train_data['preprocessed_text'].str.contains(suspicious_words)*1
train_data['text_len'] = train_data['preprocessed_text'].apply(lambda x: len(x))

test_data['money_mark'] = test_data['preprocessed_text'].str.contains(money_simbol_list)*1
test_data['suspicious_words'] = test_data['preprocessed_text'].str.contains(suspicious_words)*1
test_data['text_len'] = test_data['preprocessed_text'].apply(lambda x: len(x))


## How would work the Bag of Words with Count Vectorizer concept?

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

# Example: apply CountVectorizer on the 'text' column of the training data
vectorizer = CountVectorizer()

# Fit and transform the text data
X_bow = vectorizer.fit_transform(train_data['text'])

# Print the shape of the resulting Bag of Words matrix
print(X_bow.shape)


(5964, 79210)


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(train_data['text'])

# Print the shape of the resulting TF-IDF matrix
print(X_tfidf.shape)


(5964, 79210)


## And the Train a Classifier?

In [21]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Use TF-IDF or Bag of Words (BoW) features, depending on your choice
X = X_tfidf  # or X_bow for Bag of Words
y = train_data['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the classifier
classifier = MultinomialNB()

# Train the classifier
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Print the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Classifier accuracy: {accuracy:.2f}')


Classifier accuracy: 0.95


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code